In [2]:
import pandas as pd
from bokeh.models import ColumnDataSource, LabelSet, Title, Axis
from bokeh.plotting import figure, show
from bokeh.sampledata.sprint import sprint
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
import bokeh.io
from bokeh.io import curdoc
from bokeh.themes import Theme
from bokeh.models import CustomJS, Dropdown
from bokeh.models import PrintfTickFormatter
from bokeh.models import NumeralTickFormatter



bokeh.io.reset_output()
bokeh.io.output_notebook()
pd.set_option('display.float_format', lambda x: '%.5f' % x)

Loading BokehJS ...

In [4]:
df = pd.read_csv("../data/all_2021_nfl_contracts.csv")

In [5]:
df["contract_length"] = pd.to_numeric(df.contract_length.replace("-", None))

df["contract_expires"] = df.contract_expires.replace("-", None)
df["contract_expires"] = pd.to_numeric(df.contract_expires)

df["contract_start"] = df.contract_expires - df.contract_length
df['_contract_amount'] = df.contract_amount.map('${:,.0f}'.format)
df['_guaranteed_amount'] = pd.to_numeric(df.guaranteed_amount.replace("-",None).str.replace("$", "").str.replace(",",""))
df["contract_remaining_years"] = df.contract_expires - 2022

/tmp/ipykernel_3855768/4021721831.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['_guaranteed_amount'] = pd.to_numeric(df.guaranteed_amount.replace("-",None).str.replace("$", "").str.replace(",",""))


In [57]:
tooltips = [
    ("Team", "@tm"),
]

p = figure(width=1000, height=1000, title="Guarnteed Contract Amount vs Average Remaining Contract Years", tooltips=tooltips,)


colormap = {'chicago-bears': '#c83803'}
colors = [colormap.get(x, "white") for x in df.tm.unique()]


source = ColumnDataSource(df[(df.contract_amount > 1000000)].groupby("tm", as_index=False).mean()[["tm","contract_remaining_years", "_guaranteed_amount"]])
p.circle(source = source, x="contract_remaining_years", y="_guaranteed_amount", size=15, color="white")

p.title.text_font_size = "25px"
p.title.text_color ="#c83803"
p.title.align = 'center'
p.title.vertical_align = 'bottom'
p.min_border_top = 75
p.min_border_bottom = 75
p.yaxis.major_label_text_color = "#c83803"
p.xaxis.major_label_text_color = "#c83803"
p.background_fill_color = "#0B162A"
p.border_fill_color = "#0B162A"
p.ygrid.grid_line_alpha = 0.2
p.ygrid.grid_line_dash = [6, 4]
p.axis.axis_line_color = None
p.yaxis.major_label_text_font_size = "13pt"

p.xaxis.major_label_text_font_size = "13pt"
p.xaxis.axis_label_text_font_size = "15pt"
p.xaxis.axis_label_text_color = "#c83803"


p.yaxis.major_label_text_font_size = "13pt"
p.yaxis.axis_label_text_font_size = "15pt"
p.yaxis.axis_label_text_color = "#c83803"


p.xaxis.axis_label="Avg Remaining Contract Years"
p.yaxis.axis_label="Avg Guaranteed Contract Amount"

#p.yaxis[0].formatter = PrintfTickFormatter(format="%{:,}")
p.yaxis[0].formatter = NumeralTickFormatter(format="‘0.0a’")


#p.add_layout(labels)
 
show(p)

# Altair

In [6]:
df["remaining_guaranteed_contract_amount"] = df._guaranteed_amount / df.contract_remaining_years

In [7]:
df

,Unnamed: 0,player_name,position,age,exp,contract_amount,contract_length,guaranteed_amount,contract_expires,acquired,tm,contract_start,_contract_amount,_guaranteed_amount,contract_remaining_years,remaining_guaranteed_contract_amount
0,0,Kyler Murray,QB,25.00000,4.00000,230500000,5.00000,"$160,000,000",2029.00000,Drafted,arizona-cardinals,2024.00000,"$230,500,000",160000000.00000,7.00000,22857142.85714
1,1,Budda Baker,S,26.00000,6.00000,59000000,4.00000,"$33,100,000",2025.00000,Drafted,arizona-cardinals,2021.00000,"$59,000,000",33100000.00000,3.00000,11033333.33333
2,2,DeAndre Hopkins,WR,30.00000,9.00000,54500000,2.00000,"$60,050,000",2025.00000,Traded,arizona-cardinals,2023.00000,"$54,500,000",60050000.00000,3.00000,20016666.66667
3,3,D.J. Humphries,LT,28.00000,8.00000,51763530,3.00000,"$32,823,529",2026.00000,Drafted,arizona-cardinals,2023.00000,"$51,763,530",32823529.00000,4.00000,8205882.25000
4,4,Justin Pugh,G,32.00000,10.00000,44775000,5.00000,"$15,750,000",2023.00000,Free Agent,arizona-cardinals,2018.00000,"$44,775,000",15750000.00000,1.00000,15750000.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2903,85,Matt Cole,WR,25.00000,1.00000,705000,1.00000,-,NaN,NaN,washington-commanders,NaN,"$705,000",NaN,NaN,NaN
2904,86,Nolan Laufenberg,G,23.00000,NaN,705000,1.00000,-,NaN,Reserve/Future,washington-commanders,NaN,"$705,000",NaN,NaN,NaN
2905,87,Olubunmi Rotimi,DE,27.00000,1.00000,660000,1.00000,-,2023.00000,To Prac. Squad,washington-commanders,2022.00000,"$660,000",NaN,1.00000,NaN
2906,88,Will Bradley-King,DE,24.00000,NaN,660000,1.00000,-,2023.00000,To Prac. Squad,washington-commanders,2022.00000,"$660,000",NaN,1.00000,NaN


In [10]:
df = df[(df.contract_amount > 1000000)].groupby("tm", as_index=False).agg({"contract_remaining_years":"mean", "remaining_guaranteed_contract_amount":"sum"})[["tm","contract_remaining_years", "remaining_guaranteed_contract_amount"]]
df["Is Bears"] = df.tm == "chicago-bears"

In [11]:
df

,tm,contract_remaining_years,remaining_guaranteed_contract_amount,Is Bears
0,arizona-cardinals,2.19403,186863481.69048,False
1,atlanta-falcons,2.35211,103878384.31667,False
2,baltimore-ravens,2.65278,156226932.81667,False
3,buffalo-bills,2.36250,190564780.31190,False
4,carolina-panthers,2.43478,204934877.06667,False
5,chicago-bears,2.19118,100551996.41667,True
6,cincinnati-bengals,2.48529,97839323.35000,False
7,cleveland-browns,2.57971,224066693.66667,False
8,dallas-cowboys,2.43678,158628261.66667,False
9,denver-broncos,2.34615,205879822.16667,False


In [12]:
import altair as alt
from vega_datasets import data
alt.Chart(df, 
          width=1000, 
          height=1000, 
          title="Chicago Bears Remaining Guaranteed Contract Amount vs Average Remaining Contract Years").mark_circle(
    size=200, 
    opacity=1).encode(text="tm",
    x=alt.X('contract_remaining_years',title='Avg Contract Remaining Years',
        scale=alt.Scale(zero=False)
    ),
    y=alt.Y('remaining_guaranteed_contract_amount',title="Total Remaining Guaranteed Contract Amount",
        scale=alt.Scale(zero=False)
    ),
    color=alt.Color("Is Bears", 
                    scale=alt.Scale(domain=[True, False], range=["#c83803", "white"])
                   ),
    tooltip=[alt.Tooltip("tm", title="Team Name"), 
             alt.Tooltip("remaining_guaranteed_contract_amount", format=",.0f", title="Guaranteed Amount"), 
             alt.Tooltip("contract_remaining_years", format=",.2f", title=" Contract Remaining Years")
            ]
).configure(background='#0B162A').configure_axis(gridColor="grey", gridDash=[6,4]).configure_axisLeft(
  labelColor='#c83803',
  titleColor='#c83803',
labelFontSize=15, titleFontSize=17
).configure_axisBottom(
  labelColor='#c83803',
  titleColor='#c83803',
    labelFontSize=15, titleFontSize=17
).configure_title(color="#c83803", fontSize=25).configure_legend(labelColor="#c83803", titleColor="#c83803",                                                             
    strokeColor='gray',
    fillColor='#0B162A',
    padding=10,
    cornerRadius=10,
    orient='top-right', labelFontSize=15, titleFontSize=15)

alt.Chart(...)